<a href="https://colab.research.google.com/github/IsuruMahakumara/MovieRecommender/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from os import listdir
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
import seaborn as sns
import time


In [3]:
dir = '/content/drive/MyDrive/Daraz-Assigment/unzipped/'

file_list = listdir(dir)

file_list.remove('readme.txt')  #13 Data Files

In [89]:
movie_countries = pd.read_csv(dir+'movie_countries.dat', sep='\t', encoding='cp1252') 

movie_directors = pd.read_csv(dir+'movie_directors.dat', sep='\t', encoding='cp1252') 

movie_genres = pd.read_csv(dir+'movie_genres.dat', sep='\t', encoding='cp1252') 

movie_tags = pd.read_csv(dir+'movie_tags.dat', sep='\t', encoding='cp1252') 

tags = pd.read_csv(dir+'tags.dat', sep='\t', encoding='cp1252') 

movie_locations = pd.read_csv(dir+'movie_locations.dat', sep='\t', encoding='cp1252') 

movies = pd.read_csv(dir+'movies.dat', sep='\t', encoding='cp1252') 

user_taggedmovies_timestamps = pd.read_csv(dir+'user_taggedmovies-timestamps.dat', sep='\t', encoding='cp1252') 

user_taggedmovies = pd.read_csv(dir+'user_taggedmovies.dat', sep='\t', encoding='cp1252') 

user_ratedmovies_timestamps = pd.read_csv(dir+'user_ratedmovies-timestamps.dat', sep='\t', encoding='cp1252') 

user_ratedmovies = pd.read_csv(dir+'user_ratedmovies.dat', sep='\t', encoding='cp1252') 

movie_actors = pd.read_csv(dir+'movie_actors.dat', sep='\t', encoding='cp1252') 

In [90]:
#Wrangling movies.dat

movies_noisy_col =['imdbID', 'spanishTitle', 'imdbPictureURL', 'rtID', 'rtPictureURL']  # 
movies_rating_col = ['rtAllCriticsRating','rtAllCriticsNumReviews', 'rtAllCriticsNumFresh', 'rtAllCriticsNumRotten', 'rtAllCriticsScore','rtTopCriticsRating',
       'rtTopCriticsNumReviews', 'rtTopCriticsNumFresh','rtTopCriticsNumRotten', 'rtTopCriticsScore','rtAudienceRating',
       'rtAudienceNumRatings', 'rtAudienceScore']


#Converting object to float
for col in movies_rating_col:
  movies[col] = pd.to_numeric(movies[col], errors='coerce')

#Compute mean rating 
movies["Mean Rating"] = movies[movies_rating_col].mean(axis=1)

#Min-Max Normalize the mean Rating
from sklearn.preprocessing import MinMaxScaler
x =movies["Mean Rating"].values.reshape(10197,1)
rating_scaled = MinMaxScaler().fit_transform(x)
movies["Mean Rating"] = rating_scaled


#Removing noisy columns
movies = movies.drop(columns=movies_noisy_col)
movies = movies.drop(columns=movies_rating_col)

#Rename id to moiveID for consistency
movies.rename(columns={'id':'movieID'}, inplace=True) 



In [91]:
#movie directors.dat    movies_countries.dat

print('before director  merge', movies.shape)
movie_directors.drop(['directorName'], axis=1, inplace=True)
movies  = movies.merge(movie_directors,on='movieID')

print('before actor merge', movies.shape)

##Wrangling movie_actors.dat
movie_actors.drop(['actorName'], axis=1, inplace=True)
#Selecting the top two actors
movie_actors = movie_actors.loc[movie_actors['ranking']<3]
#Return top two actors into one field
movie_actors['actorID'] = movie_actors['actorID'] + "  "
actors = movie_actors.groupby('movieID')["actorID"].sum()
movie_actors = pd.DataFrame(actors)

#Merge with movies
movies = movies.merge(movie_actors, on = 'movieID')

print('before genre merge', movies.shape)


#Wrangling movie_genres.dat
#Returning genres to one field
movie_genres['genre'] = movie_genres['genre'] +" "
genres = movie_genres.groupby(['movieID'])['genre'].sum()
genres = pd.DataFrame(genres)
#Mergin with movies
movies = movies.merge(genres, on='movieID')

movies.drop(columns=['Mean Rating'], inplace=True)


movies.tail()


before director  merge (10197, 4)
before actor merge (10155, 5)
before genre merge (10134, 6)


,movieID,title,year,directorID,actorID,genre
10129,65088,Bedtime Stories,2008,adam_shankman,adam_sandler keri_russell,Adventure Children Comedy
10130,65091,Manhattan Melodrama,1934,ws_van_dyke,clark_gable william_powell,Crime Drama Romance
10131,65126,Choke,2008,clark_gregg,anjelicahuston sam_rockwell,Comedy Drama
10132,65130,Revolutionary Road,2008,sam_mendes,kate_winslet leonardo_di_caprio,Drama Romance
10133,65133,Blackadder Back & Forth,1999,paul_weiland,colin_firth hugh_laurie,Comedy


In [7]:
movies['year'] = movies['year'].astype('str')

movies ['Key'] = movies['directorID']+' '+movies['actorID']+' '+movies['genre']+' '+movies['year']
movies[['year', 'title', 'Key']].head()


#############ERROR############:::>>>>>(10134, 7)>>>>>>>>shape of movies

,year,title,Key
0,1995,Toy story,john_lasseter tim_allen tom_hanks Adventure...
1,1995,Jumanji,joe_johnston bonnie_hunt robin_williams Adv...
2,1993,Grumpy Old Men,donald_petrie jack_lemmon walter_matthau Co...
3,1995,Waiting to Exhale,forest_whitaker angela_bassett whitney_housto...
4,1995,Father of the Bride Part II,charles_shyer diane_keaton steve_martin Com...


In [33]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_csr = cv.fit_transform(movies['Key'])

In [95]:
##Cosine similarity
cosine_sim = cosine_similarity(X_csr)


similar_movies = list(enumerate(cosine_sim[296]))
similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
top = similar_movies[0:13]

rec = pd.DataFrame(top, columns=['movieIndex', 'Similarity'])   
Cos_Title= movies['title'].iloc[rec.movieIndex.values]
a = Cos_Title.values
print(*a, sep = "\n")
movies.iloc[rec.movieIndex.values]

The Cement Garden
Mi vida loca
Trois couleurs: Rouge
What's Eating Gilbert Grape
Little Buddha
This Boy's Life
In the Name of the Father
King of the Hill
Naked
Philadelphia
Poetic Justice
Ruby in Paradise
Rudy


,movieID,title,year,directorID,actorID,genre
1296,1437,The Cement Garden,1993,_0001949,jospeh_phd_kramer,Drama
258,269,Mi vida loca,1993,allison_anders,angel_aviles seidy_lopez,Drama
295,306,Trois couleurs: Rouge,1994,krzysztof_kieslowski,irene_jacob jeanlouis_trintignant,Drama
324,337,What's Eating Gilbert Grape,1993,162659443,johnny_depp juliette_lewis,Drama
352,365,Little Buddha,1993,bernardo_bertolucci,chris_isaak keanu_reeves,Drama
375,388,This Boy's Life,1993,michael_catonjones,ellen_barkin robert_de_niro,Drama
455,475,In the Name of the Father,1993,jim_sheridan,daniel_daylewis pete_postlethwaite,Drama
463,483,King of the Hill,1993,steven_soderbergh,jeroen_krabbe jesse_bradford,Drama
480,501,Naked,1993,mike_leigh,david_thewlis lesley_sharp,Drama
487,508,Philadelphia,1993,jonathan_demme,denzel_washington tom_hanks,Drama


In [10]:
rating =  user_ratedmovies_timestamps.pivot(index='userID',columns='movieID',values='rating')
rating.fillna(0,inplace=True)


In [11]:
#Prediction Of Rating Using Single Value Decomposition 
R = rating.values
R_mean = np.mean(R, axis = 1).reshape(-1,1)
R_demeaned = R - R_mean


from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)
predicted_R = np.dot(np.dot(U, sigma), Vt) + R_mean
predicted_R = pd.DataFrame(predicted_R, index=rating.index, columns=rating.columns)

In [12]:


def get_index(title):
    item = movies[movies.title == title]["index"].values[0]
    return print(item)


def get_title(movieID):
  title = movies['title'].loc[movies['movieID']==movieID].values[0]
  return title
    


In [102]:
sample = [ 939, 1410, 3925, 5161, 6977]
recom_Index = np.zeros(5, dtype=int)

for i in range(5):

  similar_movies =cosine_sim[sample[i]].reshape(-1,1)
  similar_movies= pd.DataFrame(similar_movies, columns=['Similarity']).sort_values(by =['Similarity'], ascending=False)[1:2].index.values
  recom_Index [i]= similar_movies
movies.iloc[recom_Index]

,movieID,title,year,directorID,actorID,genre
979,1081,Victor Victoria,1982,blake_edwards,james_garner julie_andrews,Comedy Musical Romance
5097,5459,Men in Black II,2002,barry_sonnenfeld,tommy_lee_jones will_smith,Action Comedy Sci-Fi
7602,8671,Zus & zo,2001,paula_van_der_oest,anneke_blok annet_nieuwenhuyzen,Comedy Drama Romance
5818,6218,Bend It Like Beckham,2002,gurinder_chadha,keira_knightley parminder_nagra,Adventure Comedy Drama Romance
6258,6683,Man on Fire,2004,tony_scott,dakota_fanning denzel_washington,Drama


In [120]:
recom_ID = movies.iloc[recom_Index]['movieID'].values


predicted_R[recom_ID]



movieID,1081,5459,8671,6218,6683
userID,,,,,
75,-0.026194,0.215170,-0.003609,-0.177155,-0.010647
78,0.668332,0.759107,0.001047,1.510106,0.380183
127,-0.004595,0.070571,0.003215,0.067621,-0.012369
170,0.127078,0.181688,0.002523,0.001725,0.019296
175,-0.098030,-0.201895,-0.026340,1.593062,-0.005564
...,...,...,...,...,...
71497,0.165819,1.170478,0.016644,0.917112,0.001877
71509,0.866310,0.983137,-0.021601,1.576619,0.292056
71525,0.080382,1.804823,-0.019379,1.312573,-0.018460


In [92]:
User_ratedMovies = rating.iloc[75].apply(lambda x:x!=0)
watched = User_ratedMovies[User_ratedMovies].index.values.reshape(1,-1)

User_ratedMovies.head()

movieID
1    False
2    False
3    False
4    False
5    False
Name: 2477, dtype: bool

In [38]:
#Select the 05 most recent and highest rated movies of UserID
User_ratedMovies = rating.iloc[75].apply(lambda x:x>4)
sample = User_ratedMovies[User_ratedMovies].index.values
sample = np.random.choice(sample, 5)

sample

sample_index = movies.loc[movies['movieID'].isin(sample)].index.values


sample_index


array([ 939, 1410, 3925, 5161, 6977])

In [16]:
##API returning movies 


def Recommendations(UserID):

#Select at random  05  highest rated movies seen by  UserID 
  User_ratedMovies = rating.iloc[75].apply(lambda x:x>4)
  sample = User_ratedMovies[User_ratedMovies].index.values
  sample = np.random.choice(sample, 5)


  #Nearest Neighbours
  KNN = NearestNeighbors(n_neighbors=5,metric='cosine', radius=0.4).fit(X)

  #Return 03 movieIDs for each of the 05 movies of UserID
  recs_idx = KNN.kneighbors(item,  15, return_distance=False)
  recs_idx = recs_idx.flatten()

  #Select Movie title and Output 
  Title = movies['title'].iloc[recs_idx]
  rec_movieID = movies['id'].iloc[recs_idx]
  a = Title.values
  print("Recommendations")
  print(*a, sep = "\n")


  STitle = movies['title'].iloc[movieID]
  s = STitle.values
  print("\n\nSamples Train Movies\n")
  print(*s, sep="\n")

  rec_movieID 

  return Title, rec_movieID


In [17]:
user_preference.loc[user_preference['userID']==75][0:1]

NameError: ignored

In [ ]:
Rec_Title, rec_movieID = Recommendations(75)


In [ ]:
Cos_Title

Cos_MovieID = rec['movieID'].values

np.isin(Cos_MovieID, rec_movieID)

In [ ]:
###Evaluation 

#Precision = TP /(TP+FP)
#Recall = TP / (TP+FN)

#def Precision_User(UserID, Rec_Movies):
UserID = 75
User_movies = user_preference.loc[(user_preference['userID']==UserID) & (user_preference['rating']>4)]['movieID'].values

User_movies
rec_movieID.values
Cos_MovieID

np.isin(User_movies, Cos_MovieID)#.sum()  

